In [8]:
import os
import zipfile
import bz2
import requests
from typing import Tuple
from google.cloud import storage
from datetime import timedelta
from typing import List, Tuple
from prefect import flow, task
from prefect.tasks import task_input_hash, exponential_backoff

In [2]:
def download_data(url: str, output_dir: str) -> str:

    output_dir = os.path.abspath(output_dir)

    os.makedirs(output_dir, exist_ok=True)
    file_path = os.path.join(output_dir, os.path.basename(url))

    response = requests.get(url)
    with open(file_path, 'wb') as f:
        f.write(response.content)

    return file_path

def extract_data(file_path: str, output_dir: str) -> Tuple[str, str]:

    output_dir = os.path.abspath(output_dir)

    os.makedirs(output_dir, exist_ok=True)
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)

    extracted_files = []
    for root, _, files in os.walk(output_dir):
        for file in files:
            extracted_files.append(os.path.join(root, file))

    return tuple(extracted_files)

In [14]:
extract_data(download_data('https://github.com/DataTalksClub/nyc-tlc-data/archive/refs/tags/yellow.zip', 'data'), 'data')

('/workspaces/DE_airline_on_time/data/yellow.zip',
 '/workspaces/DE_airline_on_time/data/nyc-tlc-data-yellow/README.md')

In [74]:
extracted_files = []
for root, _, files in os.walk('data'):
    for file in files:
        extracted_files.append(os.path.join(root, file))

print(tuple(extracted_files))
extracted_files = tuple(extracted_files)

('data/1987.csv.bz2', 'data/1988.csv.bz2', 'data/1989.csv.bz2', 'data/1990.csv.bz2', 'data/1991.csv.bz2', 'data/1992.csv.bz2', 'data/1993.csv.bz2', 'data/1994.csv.bz2', 'data/1995.csv.bz2', 'data/1996.csv.bz2', 'data/1997.csv.bz2', 'data/1998.csv.bz2', 'data/1999.csv.bz2', 'data/2000.csv.bz2', 'data/2001.csv.bz2', 'data/2002.csv.bz2', 'data/2003.csv.bz2', 'data/2004.csv.bz2', 'data/2005.csv.bz2', 'data/2006.csv.bz2', 'data/2007.csv.bz2', 'data/2008.csv.bz2', 'data/airports.csv', 'data/carriers.csv', 'data/plane-data.csv', 'data/variable-descriptions.csv')


In [79]:
import pandas as pd
def compress_files(extracted_files: Tuple[str, ...], output_dir: str) -> Tuple[str, str]:
    """
    This function compresses extracted CSV files in .bz2 format into .gz format and saves them in the specified output
    directory. If the CSV file is encoded with 'latin_1', it will first be decoded before being compressed. Additionally,
    any occurrences of the characters '-', 'ä', 'æ', and 'â' in the 'TailNum' column will be stripped of whitespace.

    Parameters:
    extracted_files (Tuple[str, ...]): A tuple of file paths to CSV files in .bz2 format to be compressed.
    output_dir (str): The directory in which the compressed files will be saved.

    Returns:
    Tuple[str, str]: A tuple of file paths to the compressed files in the output directory.

    """
    for file_path in extracted_files:
        # Check if the file is in .bz2 format
        if file_path.endswith('.bz2'):
            compressed_file_path = file_path[:-4] + '.gz'
            try:
                # Try to read the file using the default encoding ('utf-8')
                df = pd.read_csv(file_path)
                # Compress the file and save it in the output directory
                df.to_csv(compressed_file_path, index=False, compression='gzip')
                # Remove the original file
                os.remove(file_path)
            except UnicodeDecodeError:
                # If the default encoding fails, try reading the file using 'latin_1' encoding
                df = pd.read_csv(file_path, encoding='latin_1')
                df['TailNum'] = df['TailNum'].str.strip('-äæâ')
                # Compress the file and save it in the output directory
                df.to_csv(compressed_file_path, index=False, compression='gzip')
                # Remove the original file
                os.remove(file_path)
    
    # Collect all file paths in the output directory
    all_files = []
    for root, _, files in os.walk(output_dir):
        for file in files:
            all_files.append(os.path.join(root, file))
    
    # Return the file paths as a tuple
    return tuple(all_files)

In [45]:
df = pd.read_csv('data/2001.csv.bz2', encoding='latin_1')
df['TailNum'] = df['TailNum'].str.strip('-äæâ')

In [ ]:
df2 = pd.read_csv('data/2001.csv.bz2', encoding='latin_1')
df2['TailNum'] = df2['TailNum'].str.strip('-äæâ')

In [72]:
df2.columns

Index(['Unnamed: 0', 'Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime',
       'CRSDepTime', 'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum',
       'TailNum', 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

In [51]:
df2['TailNum'] = df2['TailNum'].value_counts()

äNKNOæ    176811
-N823A      3982
-N819A      3850
-N810A      3755
-N916D      3631
           ...  
N000A1         1
N661äâ         1
N662äâ         1
N7BäA1         1
N668äâ         1
Name: TailNum, Length: 4561, dtype: int64

In [57]:
df2['TailNum'] = df2['TailNum'].str.strip('-äæâ')

0            N700
1            N713
2            N702
3            N701
4            N768
            ...  
5967775     N914D
5967776     N913D
5967777     N910D
5967778    N906D1
5967779    N905D1
Name: TailNum, Length: 5967780, dtype: object

In [55]:
df2.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

In [48]:
df2.TailNum.value_counts

dtype('O')

In [ ]:
import pandas as pd

df = pd.read_csv('data/1987.csv.bz2')

df.head()

df.to_csv('data/1987.csv.gz', compression = "gzip")

In [13]:
def extract_individual_files(extracted_files: Tuple[str, ...]) -> Tuple[str, ...]:
    new_extracted_files = []
    for file_path in extracted_files:
        if file_path.endswith('.bz2'):
            with open(file_path, 'rb') as f:
                content = f.read()
            decompressed_content = bz2.decompress(content)
            new_file_path = file_path[:-4]  # remove .bz2 extension
            with open(new_file_path, 'wb') as f:
                f.write(decompressed_content)
            new_extracted_files.append(new_file_path)
        else:
            new_extracted_files.append(file_path)
    print((new_extracted_files))
    return (new_extracted_files)


In [16]:
extracted_files = ('data/1987.csv.bz2', 'data/1988.csv.bz2', 'data/1989.csv.bz2', 'data/1990.csv.bz2', 'data/1991.csv.bz2', 'data/1992.csv.bz2', 'data/1993.csv.bz2', 'data/1994.csv.bz2', 'data/1995.csv.bz2', 'data/1996.csv.bz2', 'data/1997.csv.bz2', 'data/1998.csv.bz2', 'data/1999.csv.bz2', 'data/2000.csv.bz2', 'data/2001.csv.bz2', 'data/2002.csv.bz2', 'data/2003.csv.bz2', 'data/2004.csv.bz2', 'data/2005.csv.bz2', 'data/2006.csv.bz2', 'data/2007.csv.bz2', 'data/2008.csv.bz2', 'data/airports.csv', 'data/carriers.csv', 'data/plane-data.csv', 'data/variable-descriptions.csv')
individual_files = extract_individual_files(extracted_files)
# compress_file(individual_files)

['data/1987.csv', 'data/1988.csv', 'data/1989.csv', 'data/1990.csv', 'data/1991.csv', 'data/1992.csv', 'data/1993.csv', 'data/1994.csv', 'data/1995.csv', 'data/1996.csv', 'data/1997.csv', 'data/1998.csv', 'data/1999.csv', 'data/2000.csv', 'data/2001.csv', 'data/2002.csv', 'data/2003.csv', 'data/2004.csv', 'data/2005.csv', 'data/2006.csv', 'data/2007.csv', 'data/2008.csv', 'data/airports.csv', 'data/carriers.csv', 'data/plane-data.csv', 'data/variable-descriptions.csv']


In [27]:
compress_file(['data/1987.csv', 'data/1988.csv', 'data/1989.csv', 'data/1990.csv', 'data/1991.csv', 'data/1992.csv', 'data/1993.csv', 'data/1994.csv', 'data/1995.csv', 'data/1996.csv', 'data/1997.csv', 'data/1998.csv', 'data/1999.csv', 'data/2000.csv', 'data/2001.csv', 'data/2002.csv', 'data/2003.csv', 'data/2004.csv', 'data/2005.csv', 'data/2006.csv', 'data/2007.csv', 'data/2008.csv', 'data/airports.csv', 'data/carriers.csv', 'data/plane-data.csv', 'data/variable-descriptions.csv'])

KeyboardInterrupt: 

In [28]:
import pandas as pd